# VacationPy
### Setup & Configuration

In [1]:
# Dependencies and Setup
from urllib.parse import urlencode, urlparse, parse_qsl
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pathlib import Path

# Google developer API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame
* Making a new directory for output files

In [2]:
# Set Path and Read CSV file to DataFrame
import_file = "cities.csv"
import_file_path = '../WeatherPy/output_data/'
pathname = os.path.join(import_file_path, import_file)
weather_data = pd.read_csv(pathname)

# Creating a New Directory
output_dir = Path('output_data')
output_dir.mkdir(exist_ok=True)

In [3]:
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Code,Country Name,GMT Date,Hemisphere
0,KICHERA,55.94,110.10,16.53,82,95,1.79,RU,Russian Federation,2020-07-15 22:38:04,Northen
1,TATEYAMA,34.98,139.87,21.00,88,75,1.00,JP,Japan,2020-07-15 22:38:04,Northen
2,ISANGEL,-19.55,169.27,25.00,83,20,0.98,VU,Vanuatu,2020-07-15 22:38:04,Southern
3,MUJIAYINGZI,42.12,118.78,24.19,53,29,2.95,CN,China,2020-07-15 22:38:04,Northen
4,MANIITSOQ,65.42,-52.90,7.33,74,83,7.23,GL,Greenland,2020-07-15 22:38:04,Northen
...,...,...,...,...,...,...,...,...,...,...,...
84,PUERTO NARINO,-3.77,-70.38,30.00,74,40,0.58,CO,Colombia,2020-07-15 22:38:13,Southern
85,SANTA ISABEL DO RIO NEGRO,-0.41,-65.02,24.89,81,47,0.83,BR,Brazil,2020-07-15 22:38:13,Southern
86,PORT ALFRED,-33.59,26.89,11.78,42,0,1.95,ZA,South Africa,2020-07-15 22:38:13,Southern
87,MIRAMAR,25.99,-80.23,31.00,69,40,2.65,US,United States,2020-07-15 22:34:42,Northen


### Set Access to GoogleMap
* Creating GoogleMapClient class object for requests

In [4]:
class GoogleMapClient(object):
    lat = None
    lng = None
    data_type ='json'
    location_query = None
    api_key=None
    
    def __init__(self, api_key=None, address_or_postal_code = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if g_key == None:
            raise Exception('Api key is required')
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        geocode_url = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {'address':self.location_query, 'key': self.api_key}
        params_url = urlencode(params)
        url = f"{geocode_url}?{params_url}"
        response = requests.get(url)
        if response.status_code not in range(200, 299):
            return {}
        latlng = {}
        try:
            latlng = response.json()['results'][0]['geometry']['location']
        except:
            pass
        lat, lng = latlng.get('lat'), latlng.get('lng')
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword='Mexican food', radius = 1000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = extract_lat_lng()
        end_point = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}'
        params = {
                'key': self.api_key,
                'location': f"{lat},{lng}",
                'radius': radius,
                'keyword': keyword
                        }
        params_encoded = urlencode(params)
        places_endpoint = f"{end_point}?{params_encoded}"
        response = requests.get(places_endpoint)
        if response.status_code not in range(200, 299):
            return {}
        return response.json()
    
    def detail(self, place_id='ChIJv1DlfKQsDogRhm8x60TVJyo'):
        detail_end_point = f'https://maps.googleapis.com/maps/api/place/details/{self.data_type}'
        detail_params = {
                'place_id': f'{place_id}',
                'fields': 'name,rating,formatted_phone_number,formatted_address',
                'key': self.api_key
                }
        detail_params_encoded = urlencode(detail_params)
        detail_places_endpoint = f"{detail_end_point}?{detail_params_encoded}"
        response = requests.get(detail_places_endpoint)
        if response.status_code not in range(200, 299):
            return {}
        return response.json()      

In [5]:
client = GoogleMapClient(api_key = g_key, address_or_postal_code = '225 N Columbus Dr, Chicago, IL 60601, USA')
print(client.lat, client.lng)

41.886543 -87.61998


In [6]:
client.search('International airport', radius=25000)['results'][0]['name']

"O'Hare International Airport"

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
